In [ ]:
import requests
import pyotp
import time

base_url = "http://localhost:14374"
failures = 0


def register_user():
    login_response = requests.post(
        f"{base_url}/register",
        json={
            "email": "test@test.com",
            "first_name": "Test",
            "last_name": "Name",
            "company_name": "Test Company",
            "job_title": "Test Job Title",
        },
    )
    response = login_response.json()
    if "mfa_token" in response:
        mfa_token = response["mfa_token"]
        totp = pyotp.TOTP(mfa_token)
        return f"Success! Your MFA token is: {totp.now()}"
    else:
        return response


while failures < 100:
    try:
        print(register_user())
        break
    except Exception as e:
        print(e)
        failures += 1
        time.sleep(5)

In [12]:
import uuid

print(uuid.uuid4())

e682d6e9-9405-4f39-a1aa-7e09574a6703


In [14]:
import os
from hashlib import md5
from Crypto.Cipher import AES
import base64


def encrypt(passphrase, data):
    passphrase = passphrase.encode("utf-8")
    salt = os.urandom(8)
    passphrase += salt
    key = md5(passphrase).digest()
    final_key = key
    while len(final_key) < 32 + 16:
        key = md5(key + passphrase).digest()
        final_key += key
    key_iv = final_key[: 32 + 16]
    key = key_iv[:32]
    iv = key_iv[32:]
    aes = AES.new(key, AES.MODE_CBC, iv)
    padded_data = data.encode("utf-8") + (16 - len(data) % 16) * bytes(
        [16 - len(data) % 16]
    )
    encrypted_data = aes.encrypt(padded_data)
    encrypted = b"Salted__" + salt + encrypted_data
    return base64.b64encode(encrypted).decode("utf-8")


def decrypt(passphrase, data):
    try:
        passphrase = passphrase.encode("utf-8")
        encrypted = base64.b64decode(data)
        assert encrypted[0:8] == b"Salted__"
        salt = encrypted[8:16]
        assert len(salt) == 8, len(salt)
        passphrase += salt
        key = md5(passphrase).digest()
        final_key = key
        while len(final_key) < 32 + 16:
            key = md5(key + passphrase).digest()
            final_key += key
        key_iv = final_key[: 32 + 16]
        key = key_iv[:32]
        iv = key_iv[32:]
        aes = AES.new(key, AES.MODE_CBC, iv)
        data = aes.decrypt(encrypted[16:])
        decrypted = data[: -(data[-1] if type(data[-1]) == int else ord(data[-1]))]
        return decrypted.decode("utf-8")
    except:
        return encrypted


passphrase = "None of your business"
data = "Hello, World!"

encrypted = encrypt(passphrase=passphrase, data=data)
print(encrypted)

U2FsdGVkX1/OX3r9OGS6V2bFxOMNiA/CIQr68ZPv4CA=


In [15]:
decrypted = decrypt(passphrase=passphrase, data=encrypted)
print(decrypted)

Hello, World!
